# Tutorial 3

**In this tutorial you will:**
- Build a Chatbot connected to a code interpreter able to execute Python code [Section 1](#1-chatbotwithinterpreter-expanding-interactivity-and-functionality)

In [1]:
%load_ext autoreload
%autoreload 2
#imports
from aiflows.utils.general_helpers import read_yaml_file, quick_load_api_keys
from aiflows.backends.api_info import ApiInfo
from aiflows.utils import serve_utils
from aiflows.utils import colink_utils
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
from aiflows import flow_verse
import sys
import os
sys.path.append("..")
from utils import compile_and_writefile, dict_to_yaml
import json
import copy
#Specify path of your flow modules


/Users/nicolasbaldwin/opt/miniconda3/envs/mockenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#starting a local colink server
cl = colink_utils.start_colink_server()

In [3]:
# Start Worker thread
run_dispatch_worker_thread(cl)

[2024-03-22 11:44:17,869][aiflows.workers.dispatch_worker:220][INFO] - Dispatch worker started in attached thread.
[2024-03-22 11:44:17,871][aiflows.workers.dispatch_worker:221][INFO] - dispatch_point: coflows_dispatch


In [4]:
# Start 2nd Worker thread (in case you're making blocking calls)
run_dispatch_worker_thread(cl)

[2024-03-22 11:44:19,111][aiflows.workers.dispatch_worker:220][INFO] - Dispatch worker started in attached thread.
[2024-03-22 11:44:19,114][aiflows.workers.dispatch_worker:221][INFO] - dispatch_point: coflows_dispatch


## 1. ChatBotWithInterpreter: Expanding Interactivity and Functionality

In this section, we'll explore the implementation of a chatbot integrated with a code interpreter, offering new dimensions of interactivity and functionality. By connecting an interpreter capable of compiling and executing code to the chatbot, we unlock numerous benefits. This setup enables users to prompt the chatbot with specific tasks, allowing them to provide instructions like "download Apple's stock prices, plot them, and save them in a PDF." In response, the chatbot generates tailored code to execute these tasks, and executes it. 

Additionally, integrating the interpreter enhances the chatbot's adaptability and versatility. By understanding the user's environment, such as their Python environment, the chatbot can execute code seamlessly within that context. This capability extends to installing necessary Python packages on-the-fly, ensuring smooth execution without requiring user intervention. Such integration enhances the chatbot's utility, making it a versatile tool for coding assistance, troubleshooting, and rapid prototyping. Furthermore, this integration ensures that all generated code is compilable, maintaining a seamless user experience and efficient task execution.


### 1.1 Leveraging ChatAtomicFlow to write a CodeGenerator Flow

##### Pulling ChatAtomicFlow from FlowVerse

In [5]:
#Install dependencies from FlowVerse
dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "main"},
]
from aiflows import flow_verse
flow_verse.sync_dependencies(dependencies)

[2024-03-22 11:44:22,807][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatWithCompiler/flow_modules...
[2024-03-22 11:44:22,811][aiflows.flow_verse.loading:563][INFO] - aiflows/ChatFlowModule:coflows will be fetched from remote


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 70295.60it/s]


[2024-03-22 11:44:23,854][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatWithCompiler/flow_modules/aiflows/ChatFlowModule']

##### Writing the CodeGenerator's Default Config

In [6]:
#Defining prompt templates

#system prompt template
system_prompt_template = \
"""
You are a world class programmer that can complete any goal with code.
      
Your tasks are:
    1. Write a python code in order to achieve a goal that is given to you. This code should be able to be run by the executor.

Notice that:
    1. If you use any external libraries, you must install them withing the code (in python)
    2. All functions you write are modular, so make sure you make imports necessary imports within the function.
    3. You must write docstrings for every function you write.

    
Your function will then be imported and called by an executor to finish the goal, you do not need to worry about the execution part.
The executor will give you feedback on the code you write, and you can revise your code based on the feedback.

Performance Evaluation:
1. You must write your code in python
2. Your answer must be able to be compiled and run by the executor (i.e, do not write codeblocks)

**It's important that you should only respond in JSON format as described below:**
Response Format:
{
    "language_of_code": "language of the code",
    "code": "String of the code and docstrings corresponding to the goal",
    "finish": "True if you have finished the goal, False otherwise. Take in consideration the feedback you receive to decide if you have finished the goal or not",
}
Ensure your responses can be parsed by Python json.loads
    
**It's important that the code you generate can be written by Python write, and is human-readable. The written file must also be indented and formatted, so that it is human-readable.**
"""

#init_human_prompt_template
#This is the prompt passed to the ChatAtomicFlow the first time its called (i.e., the user asks to accomplish a goal)
init_human_prompt_template = \
"""
"Here is the goal you need to achieve:
{{goal}}"
"""

#human_prompt_template
#This is the prompt passed to the ChatAtomicFlow after the first time its called (i.e., feedback from the code interpreter)
human_prompt_template = \
"""
Here is the goal you need to achieve:
{{goal}}
Here is the previous code you have written:
{{previous_code}}
Here is the feedback from the previous code:
{{feedback}}
"""

In [7]:
default_config_code_generator = {
    "_target_": "CodeGeneratorFlowModule.CodeGenerator.CodeGenerator.instantiate_from_default_config",
    "name": "CodeGenerator",
    "description": "Writes code with given instruction",
    "backend": {
        "api_infos": "???"
    },
    "model_name": {
        "openai": "gpt-4"
    },
    "input_interface_non_initialized": ["goal"],

    "input_interface_initialized": ["goal", "previous_code", "feedback"],

    "output_interface": ["language_of_code", "code"],

    "system_message_prompt_template": {
        "template": f'{system_prompt_template}'
    },
  
    "human_message_prompt_template": {
        "template": f'{human_prompt_template}',
        "input_variables": ["goal", "previous_code", "feedback"],
    },

    "init_human_message_prompt_template": {
        "template": f'{init_human_prompt_template}',
        "input_variables": ["goal"]
    },
    #keep as context window only the first message (system prompt template) and the last two messages (last user-assistant interaction)
    "previous_messages": {
        "first_k": 1,
        "last_k": 2,
    }
}
dict_to_yaml(default_config_code_generator, "CodeGeneratorFlowModule/CodeGenerator.yaml")

##### Writing the CodeGenerator Flow

**Requirements:** We want to make sure that the CodeGenerator generates a json parsable output. Implement this in the `CodeGenerator` class by requerying the LLM if it doesn't generate a parsable output (specifying to reformat its previous output).

In [8]:
%%compile_and_writefile CodeGeneratorFlowModule/CodeGenerator.py

from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
from flow_modules.aiflows.ChatFlowModule import ChatAtomicFlow
import json


class CodeGenerator(ChatAtomicFlow):

    def run(self, input_message: FlowMessage):
        input_data = input_message.data
        json_parsable = False
        response = None
        
        #ensure the response is json parsable
        while not json_parsable:
            
            output = self.query_llm(input_data=input_data).strip()
            
            try:
                response = json.loads(output)
                json_parsable = True
            
            except (json.decoder.JSONDecodeError, json.JSONDecodeError):
                
                feedback = "The previous response cannot be parsed with json.loads, it \
                    could be the backslashes used for escaping single quotes in the string arguments of the code are not properly \
                        escaped themselves within the JSON context. Next time, do not provide any comments or code blocks. \
                            Make sure your next response is purely json parsable."
                previous_code = output
                new_input_data = input_data.copy()
                new_input_data = {
                    "goal": input_data["goal"],
                    "feedback": feedback,
                    "previous_code": previous_code,   
                }
                input_data = new_input_data

        
        reply = self.package_output_message(
            input_message = input_message,
            response = response
        )
        self.send_message(reply)
        

##### Serving the CodeGenerator Flow:

In [9]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="CodeGeneratorFlowModule.CodeGenerator.CodeGenerator",
    flow_endpoint="CodeGenerator",
)

[2024-03-22 11:44:36,407][aiflows.utils.serve_utils:116][INFO] - Started serving CodeGeneratorFlowModule.CodeGenerator.CodeGenerator at flows:CodeGenerator.
[2024-03-22 11:44:36,410][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 11:44:36,413][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-22 11:44:36,415][aiflows.utils.serve_utils:119][INFO] - singleton: False



True

##### Testing the CodeGenerator Flow:

We will now test the CodeGenerator Flow by prompting the chatbot with a task and observing the generated code. We will use the following task as an example: "Download Apple's stock prices, plot them, and save them in a PDF." The chatbot should generate code that accomplishes this task, and we will evaluate the generated code's quality and correctness. To showcase the limitations of using the code generator on its own, we will show that the code is not always compilable in your Python environment. To showcase we will manually uninstall, if you do have it installed, the `yfinance` package from the Python environment (which is a library the Flow will use in its answer)

In [13]:
!pip uninstall yfinance -y

Found existing installation: yfinance 0.2.37
Uninstalling yfinance-0.2.37:
  Successfully uninstalled yfinance-0.2.37


In [15]:

path_to_yaml = os.path.join("CodeGeneratorFlowModule", "CodeGenerator.yaml")
default_config = read_yaml_file(path_to_yaml)
overrides = copy.deepcopy(default_config)
api_information = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(overrides, api_information)

proxy_code_generator = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="CodeGenerator", #TODO: SPECIFY THE ENDPOINT
    user_id="local",
    config_overrides=overrides,
)
data = {
    "goal": "Download Apple's stock price between 2015 and 2016, make a plot with it and save it as a pdf in './apple_stocks.pdf'",
}
input_message = proxy_code_generator.package_input_message(data)
future = proxy_code_generator.get_reply_future(input_message)
reply_data = future.get_data()
print("Data sent:\n",  data, "\n")
print(f'REPLY:\n{reply_data["code"]} \n')


[2024-03-22 11:46:28,640][aiflows.utils.serve_utils:336][INFO] - Mounted 4d70e252-ca90-4483-8e1e-d5efd4821766 at flows:CodeGenerator:mounts:local:4d70e252-ca90-4483-8e1e-d5efd4821766


[2024-03-22 11:46:28,723][aiflows.workers.dispatch_worker:113][INFO] - 
~~~ Dispatch task ~~~
[2024-03-22 11:46:28,730][aiflows.workers.dispatch_worker:155][INFO] - flow_endpoint: CodeGenerator
[2024-03-22 11:46:28,731][aiflows.workers.dispatch_worker:156][INFO] - flow_id: 4d70e252-ca90-4483-8e1e-d5efd4821766
[2024-03-22 11:46:28,732][aiflows.workers.dispatch_worker:157][INFO] - owner_id: local
[2024-03-22 11:46:28,734][aiflows.workers.dispatch_worker:158][INFO] - message_paths: ['push_tasks:42c6eadb-af55-4b04-b284-15282c340f41:msg']
[2024-03-22 11:46:28,739][aiflows.workers.dispatch_worker:159][INFO] - parallel_dispatch: False

[2024-03-22 11:46:28,816][aiflows.workers.dispatch_worker:182][INFO] - Input message source: Proxy_CodeGenerator
[2024-03-22 11:46:48,398][aiflows.workers.dispatch_worker:113][INFO] - 
~~~ Dispatch task ~~~
[2024-03-22 11:46:48,407][aiflows.workers.dispatch_worker:155][INFO] - flow_endpoint: InterpreterAtomicFlow
[2024-03-22 11:46:48,410][aiflows.workers.dispat

Data sent:
 {'goal': "Download Apple's stock price between 2015 and 2016, make a plot with it and save it as a pdf in './apple_stocks.pdf'"} 

REPLY:
```python
import yfinance as yf
import matplotlib.pyplot as plt


def download_and_plot_stock_price():
    '''
    Download Apple's stock price between 2015 and 2016, make a plot with it, and save it as a pdf.
    '''
    # Download Apple's stock price data
    apple_data = yf.download('AAPL', start='2015-01-01', end='2016-12-31')
    
    # Plot the 'Close' price
    plt.figure(figsize=(12, 6))
    plt.plot(apple_data['Close'])
    plt.title('Apple Stock Price (2015 - 2016)')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    
    # Save the plot as a pdf
    plt.savefig('./apple_stocks.pdf')
    
    return 'Plot saved as apple_stocks.pdf'

# Call the function to download, plot, and save the stock price
download_and_plot_stock_price()
``` 



Now if you copy paste the reply from the chatbot, you will see that the code is not compilable. This is because the `yfinance` package is not installed in your Python environment. This is an example where the interpreter comes in handy.

In [16]:
##copy paste reply by code generator and try to run code HER
import yfinance as yf
import matplotlib.pyplot as plt


def download_and_plot_stock_price():
    '''
    Download Apple's stock price between 2015 and 2016, make a plot with it, and save it as a pdf.
    '''
    # Download Apple's stock price data
    apple_data = yf.download('AAPL', start='2015-01-01', end='2016-12-31')
    
    # Plot the 'Close' price
    plt.figure(figsize=(12, 6))
    plt.plot(apple_data['Close'])
    plt.title('Apple Stock Price (2015 - 2016)')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    
    # Save the plot as a pdf
    plt.savefig('./apple_stocks.pdf')
    
    # Show the plot
    plt.show()


# Call the function to download, plot, and save Apple's stock price
if __name__ == '__main__':
    download_and_plot_stock_price()

ModuleNotFoundError: No module named 'yfinance'

### 1.2 Using the Interperter Flow Module

We already have an interpreter flow module implemented on the flow verse which you can checkout [here](https://huggingface.co/aiflows/InterpreterFlowModule) !

Let's check out its config, serve it and test it.

##### Pulling the InterpreterFlowModule from FlowVerse



In [17]:
#Install dependencies from FlowVerse
dependencies = [
    {"url": "aiflows/InterpreterFlowModule", "revision": "main"},
]
from aiflows import flow_verse
flow_verse.sync_dependencies(dependencies)


[2024-03-22 11:46:37,577][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatWithCompiler/flow_modules...
[2024-03-22 11:46:37,581][aiflows.flow_verse.loading:563][INFO] - aiflows/InterpreterFlowModule:coflows will be fetched from remote


Fetching 22 files: 100%|██████████| 22/22 [00:00<00:00, 8087.89it/s]


[2024-03-22 11:46:38,138][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatWithCompiler/flow_modules/aiflows/ChatFlowModule',
 '/Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatWithCompiler/flow_modules/aiflows/InterpreterFlowModule']

In [18]:
#print it's default config

interpreter_default_config = read_yaml_file("flow_modules/aiflows/InterpreterFlowModule/InterpreterAtomicFlow.yaml")
print(json.dumps(interpreter_default_config, indent=4))

{
    "name": "InterpreterAtomicFlow",
    "description": "A flow that compiles and runs codes",
    "_target_": "flow_modules.aiflows.InterpreterFlowModule.InterpreterAtomicFlow.instantiate_from_default_config",
    "max_output": 2000,
    "input_interface": [
        "language",
        "code"
    ],
    "output_interface": [
        "interpreter_output"
    ]
}


##### Serving the InterpreterFlowModule

In [19]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.InterpreterFlowModule.InterpreterAtomicFlow",
    flow_endpoint="InterpreterAtomicFlow",
)
run_dispatch_worker_thread(cl)

[2024-03-22 11:46:43,341][aiflows.utils.serve_utils:116][INFO] - Started serving flow_modules.aiflows.InterpreterFlowModule.InterpreterAtomicFlow at flows:InterpreterAtomicFlow.
[2024-03-22 11:46:43,342][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 11:46:43,343][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-22 11:46:43,345][aiflows.utils.serve_utils:119][INFO] - singleton: False

[2024-03-22 11:46:43,347][aiflows.workers.dispatch_worker:220][INFO] - Dispatch worker started in attached thread.
[2024-03-22 11:46:43,363][aiflows.workers.dispatch_worker:221][INFO] - dispatch_point: coflows_dispatch


##### Testing the InterpreterFlowModule

In [20]:
proxy_code_interpreter = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="InterpreterAtomicFlow", #TODO: SPECIFY THE ENDPOINT
    user_id="local",
)
data = [
    #should return "Hello, World!"
    {"language": "Python", "code": "print('Hello, World!')"},
    #should return an error
    {"language": "Python", "code": "import yfinance as yf\nyf.download('AAPL', start='2015-01-01', end='2016-01-01')"},
]

for dp in data:
    input_message = proxy_code_interpreter.package_input_message(dp)
    future = proxy_code_interpreter.get_reply_future(input_message)
    reply_data = future.get_data()
    print("Data sent:\n",  dp, "\n")
    print("REPLY:\n", reply_data, "\n")

[2024-03-22 11:46:46,988][aiflows.utils.serve_utils:336][INFO] - Mounted 4f934a31-c926-47f9-9205-7837c2adc4d9 at flows:InterpreterAtomicFlow:mounts:local:4f934a31-c926-47f9-9205-7837c2adc4d9


Data sent:
 {'language': 'Python', 'code': "print('Hello, World!')"} 

REPLY:
 {'interpreter_output': 'Hello, World!'} 

Data sent:
 {'language': 'Python', 'code': "import yfinance as yf\nyf.download('AAPL', start='2015-01-01', end='2016-01-01')"} 

REPLY:
 {'interpreter_output': 'File "<stdin>", line 4, in <module>\nModuleNotFoundError: No module named \'yfinance\''} 



### 1.3 Combining the CodeGenerator and Interpreter Flow Modules (ChatCodeInterpreter Flow)

Now that we have both the CodeGenerator and Interpreter Flow Modules, we can combine them to create a ChatCodeInterpreter Flow. This flow will leverage the CodeGenerator to generate code based on user prompts, and the Interpreter to execute the generated code. The CodeGenerator will decide when to finish generating the code and return the final output (see system prompt template)

##### Writing the default config of the ChatCodeInterpreter Flow

In [21]:
init_human_prompt_template = \
"""
Here is the goal you need to achieve:
{{goal}}"
Feedback:
Do not finish this goal yet. You will receive feedback on the code you write.
"""


In [22]:
default_config_ChatCodeInterpreter = \
{
    "name": "ChatCodeInterpreter",
    "description": "A code which generate code with the help of a interpreter",

    # TODO: Define the target
    "_target_": "ChatCodeInterpreterFlowModule.ChatCodeInterpreter.ChatCodeInterpreter.instantiate_from_default_config",

    "input_interface": "goal",
    "output_interface": ["code","interpreter_output"],
    
    "subflows_config": {
        "Coder": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_endpoint": "CodeGenerator",
            "name": "Proxy of Coder",
            "description": "A proxy flow of the Coder flow.",
             "backend": {
                "api_infos": "???",
                "model_name": {"openai": "gpt-4"}
            },
             "init_human_message_prompt_template": {
                "template": f'{init_human_prompt_template}',
                "input_variables": ["goal"]
             }
        },
        "Interpreter": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
           
            "user_id": "local",
            
            "flow_endpoint": "InterpreterAtomicFlow",
            "name": "Proxy of Interpreter Flow",
            "description": "A proxy flow of the Interpreter Flow.",
        }
    }
}
dict_to_yaml(default_config_ChatCodeInterpreter, "ChatCodeInterpreterFlowModule/ChatCodeInterpreter.yaml")

##### Writing the ChatCodeInterpreter Flow


In [23]:
%%compile_and_writefile ChatCodeInterpreterFlowModule/ChatCodeInterpreter.py

from aiflows.base_flows import CompositeFlow
from aiflows.messages import FlowMessage
from aiflows.interfaces import KeyInterface

class ChatCodeInterpreter(CompositeFlow):
        
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.input_interface_generate_reply = KeyInterface(
            keys_to_rename={"question": "prompt"},
            keys_to_select=["code", "interpreter_output"]
        )
        
        self.first_input_interface_coder = KeyInterface(
            keys_to_select=["goal"]
        )
        
        self.input_interface_coder = KeyInterface(
            keys_to_rename={"code": "previous_code", "interpreter_output": "feedback"},
            keys_to_select=["goal", "previous_code", "feedback"]
        )
        
        self.input_interface_interpreter = KeyInterface(
            keys_to_rename={"code": "code", "language_of_code": "language"},
            keys_to_select=["code", "language"]
        )
        
    def set_up_flow_state(self):
        super().set_up_flow_state()
        self.flow_state["previous_state"] = None

    def determine_current_state(self):
        previous_state = self.flow_state["previous_state"]
        
        if previous_state is None:
            return "Coder"
        
        elif previous_state == "Coder":
            if self.flow_state["finish"]:
                return "GenerateReply"
            return "Interpreter"
            
        elif previous_state == "Interpreter":
            return "Coder"
        
        elif "GenerateReply":
            return None
        
        else:
            raise ValueError(f"Invalid state: {previous_state}")
                        
    def call_coder(self):
        
        if self.flow_state["previous_state"] is None:
            input_interface = self.first_input_interface_coder
        else:
            input_interface = self.input_interface_coder
        
        message = self.package_input_message(
            data = input_interface(self.flow_state),
            dst_flow = "Coder"
        )
        self.subflows["Coder"].get_reply(
            message,
        )
        
    def call_interpreter(self):
        
        input_interface = self.input_interface_interpreter
        
        message = self.package_input_message(
            data = input_interface(self.flow_state),
            dst_flow = "Interpreter"
        )
        
        self.subflows["Interpreter"].get_reply(
                message,
        )
        
    def generate_reply(self):
          
        input_interface = self.input_interface_generate_reply
          
        reply = self.package_output_message(
            input_message=self.flow_state["initial_message"],
            response=input_interface(self.flow_state),
        )
        self.send_message(reply)
        
    def register_data_to_state(self, input_message):
        
        previous_state = self.flow_state["previous_state"]
        
        #first call to flow
        if previous_state is None:
            #register initial message so we can reply to it later
            self.flow_state["initial_message"] = input_message
            #register the question
            self.flow_state["goal"] = input_message.data["goal"]
        
        elif previous_state == "Coder":
            self.flow_state["code"] = input_message.data["code"]
            self.flow_state["language_of_code"] = input_message.data["language_of_code"]
            self.flow_state["finish"] = input_message.data["finish"]
        
        #case where our last call was to the safeguard
        elif previous_state == "Interpreter":
            self.flow_state["interpreter_output"] = input_message.data["interpreter_output"]

   
    def run(self, input_message: FlowMessage):
        self.register_data_to_state(input_message)
        
        current_state = self.determine_current_state()
        
        if current_state == "Coder":
            self.call_coder()
            
        elif current_state == "Interpreter":
            self.call_interpreter()
            
        elif current_state == "GenerateReply":
            self.generate_reply()
        
        self.flow_state["previous_state"] = current_state if current_state != "GenerateReply" else None

##### Serve the ChatCodeInterpreter Flow

In [24]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ChatCodeInterpreterFlowModule.ChatCodeInterpreter.ChatCodeInterpreter",
    flow_endpoint="ChatCodeInterpreter",
)

run_dispatch_worker_thread(cl)

[2024-03-22 11:46:59,295][aiflows.utils.serve_utils:116][INFO] - Started serving ChatCodeInterpreterFlowModule.ChatCodeInterpreter.ChatCodeInterpreter at flows:ChatCodeInterpreter.
[2024-03-22 11:46:59,298][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 11:46:59,298][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-22 11:46:59,299][aiflows.utils.serve_utils:119][INFO] - singleton: False

[2024-03-22 11:46:59,301][aiflows.workers.dispatch_worker:220][INFO] - Dispatch worker started in attached thread.
[2024-03-22 11:46:59,308][aiflows.workers.dispatch_worker:221][INFO] - dispatch_point: coflows_dispatch


In [30]:
#make sure we don't have yfinacne installed
!pip uninstall yfinance -y

##### Testing the ChatCodeInterpreter Flow

Now let's test the ChatCodeInterpreter Flow by prompting the chatbot with the same task as before: "Download Apple's stock prices, plot them, and save them in a PDF." The chatbot should generate code that accomplishes this task, and the interpreter should execute the generated code (including installing the necessary packages).

In [31]:

default_config = default_config_ChatCodeInterpreter
overrides = copy.deepcopy(default_config)
api_information = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(overrides, api_information)

proxy_code_generator = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="ChatCodeInterpreter", #TODO: SPECIFY THE ENDPOINT
    user_id="local",
    config_overrides=overrides,
)
data = [
    {"id": 0, "goal": "Download Apple's stock price between 2015 and 2016, make a plot with it, \
        and save it as a pdf in Download Apple's stock price between 2015 and 2016, make a plot with \
            it and save it as a pdf in 'apple_stocks.pdf'"},
]

for dp in data:
    input_message = proxy_code_generator.package_input_message(dp)
    future = proxy_code_generator.get_reply_future(input_message)
    reply_data = future.get_data()
    print("Data sent:\n",  dp, "\n")
    print("REPLY:\n", reply_data, "\n")

[2024-03-22 11:49:33,019][aiflows.utils.serve_utils:336][INFO] - Mounted a6fafba2-7044-4a81-ba3d-c14db9ffa86f at flows:CodeGenerator:mounts:local:a6fafba2-7044-4a81-ba3d-c14db9ffa86f
[2024-03-22 11:49:33,068][aiflows.utils.serve_utils:336][INFO] - Mounted 52ef45f5-daa5-44e8-81ad-0282fd616bca at flows:InterpreterAtomicFlow:mounts:local:52ef45f5-daa5-44e8-81ad-0282fd616bca
[2024-03-22 11:49:33,107][aiflows.utils.serve_utils:336][INFO] - Mounted 0d6db9e9-c926-46d9-bb9e-337a61805cfd at flows:ChatCodeInterpreter:mounts:local:0d6db9e9-c926-46d9-bb9e-337a61805cfd


Data sent:
 {'id': 0, 'goal': "Download Apple's stock price between 2015 and 2016, make a plot with it,         and save it as a pdf in Download Apple's stock price between 2015 and 2016, make a plot with             it and save it as a pdf in 'apple_stocks.pdf'"} 

REPLY:
 {'code': 'import os\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\ntry:\n    import yfinance as yf\nexcept ImportError:\n    import pip\n    pip.main([\'install\', \'yfinance\'])\n    import yfinance as yf\n\n\ndef download_and_plot_apple_stock(start_date, end_date, output_file):\n    """\n    This function downloads Apple\'s stock price between the given dates, plots it and saves the plot as a pdf.\n    \n    Parameters:\n    start_date (str): Start date in the format \'YYYY-MM-DD\'.\n    end_date (str): End date in the format \'YYYY-MM-DD\'.\n    output_file (str): The name of the output file where the plot will be saved.\n    \n    Returns:\n    None\n    """\n    # Download Apple\'s stock price data\n 

Note how a pdf named `apple_stock_prices.pdf` is saved in the current directory. This is the pdf that was generated by the ChatAtomicFlow executerd by the interpreter Flow.

You can also check out the code it generated:

In [32]:
print(reply_data["code"])

import os
import matplotlib.pyplot as plt
import pandas as pd

try:
    import yfinance as yf
except ImportError:
    import pip
    pip.main(['install', 'yfinance'])
    import yfinance as yf


def download_and_plot_apple_stock(start_date, end_date, output_file):
    """
    This function downloads Apple's stock price between the given dates, plots it and saves the plot as a pdf.
    
    Parameters:
    start_date (str): Start date in the format 'YYYY-MM-DD'.
    end_date (str): End date in the format 'YYYY-MM-DD'.
    output_file (str): The name of the output file where the plot will be saved.
    
    Returns:
    None
    """
    # Download Apple's stock price data
    data = yf.download('AAPL', start=start_date, end=end_date)
    
    # Plot the 'Close' column
    plt.figure(figsize=(10, 5))
    plt.plot(data['Close'])
    plt.title('Apple Stock Price')
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.grid(True)
    
    # Save the plot as a pdf
    plt.savefig(output_f

If you copy paste the code in a new cell it should also run since it installed the necessary packages !

In [33]:
import os
import matplotlib.pyplot as plt
import pandas as pd

try:
    import yfinance as yf
except ImportError:
    import pip
    pip.main(['install', 'yfinance'])
    import yfinance as yf


def download_and_plot_apple_stock(start_date, end_date, output_file):
    """
    This function downloads Apple's stock price between the given dates, plots it and saves the plot as a pdf.
    
    Parameters:
    start_date (str): Start date in the format 'YYYY-MM-DD'.
    end_date (str): End date in the format 'YYYY-MM-DD'.
    output_file (str): The name of the output file where the plot will be saved.
    
    Returns:
    None
    """
    # Download Apple's stock price data
    data = yf.download('AAPL', start=start_date, end=end_date)
    
    # Plot the 'Close' column
    plt.figure(figsize=(10, 5))
    plt.plot(data['Close'])
    plt.title('Apple Stock Price')
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.grid(True)
    
    # Save the plot as a pdf
    plt.savefig(output_file, format='pdf')
    plt.close()

# Call the function
start_date = '2015-01-01'
end_date = '2016-12-31'
output_file = 'apple_stocks.pdf'
download_and_plot_apple_stock(start_date, end_date, output_file)

[*********************100%%**********************]  1 of 1 completed
